In [2]:
# Test the dataset

from lircst_ana_dataset import LircstAnaDataset 

def test_dataset():
    dataset = LircstAnaDataset('/home/samnub/dev/lircst-ana/data/')
    print(len(dataset))
    phan, sino, phantom_id = dataset[0]
    print(phan.shape, sino.shape, phantom_id)

test_dataset()

2075
(2, 128, 128) (128, 200, 100) 1742413505


In [ ]:
# Fine-tune ControlNet using pytorch-lightning on our dataset

import gc
gc.collect()

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/samnub/dev/lircst-diffusion/src/ControlNet')

from ControlNet.cldm.model import create_model, load_state_dict
from ControlNet.cldm.logger import ImageLogger
from torch.utils.data import DataLoader
from torchvision import transforms
import pytorch_lightning as pl

from ControlNet.tutorial_dataset import MyDataset
from lircst_ana_dataset import LircstAnaDataset

# Configs
resume_path = '/home/samnub/dev/lircst-diffusion/models/control_sd21_ini.ckpt'
batch_size = 4 # TODO up from 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('/home/samnub/dev/lircst-diffusion/models/cldm_v21.yaml')#.cpu()
model.load_state_dict(load_state_dict(resume_path, location='cuda'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
transform_phan = transforms.Compose([transforms.ToTensor(),
                                     ])
transform_sino = transforms.Compose([transforms.ToTensor(),
                                     ])
# dataset = LircstAnaDataset(data_dir='/home/samnub/dev/lircst-ana/data/')
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=8, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(accelerator='gpu', gpus=1, precision=32, callbacks=[logger])


# Train!
trainer.fit(model, dataloader)

: 